In [1673]:
# Import SemanticScholar (https://github.com/danielnsilva/semanticscholar)
from semanticscholar import SemanticScholar
import pandas as pd
import numpy as np
import json
import requests

In [1674]:
# Alle Zeilen und Spalten des DataFrames anzeigen
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
# np.set_printoptions(threshold=sys.maxsize)

In [1675]:
ss_completed_flag = False
am_completed_flag = False
ieee_completed_flag = False

### Frage die Graph API von SemanticScholar.org ab: </br> https://api.semanticscholar.org/api-docs/graph#tag/Paper-Data/operation/get_graph_get_paper_search

In [1676]:
# paper_doi = '10.1257/AER.98.4.1413'
# paper_doi = '10.1016/c2009-0-19715-5'
paper_doi = '10.1109/TPAMI.2016.2644615'

req = requests.get("https://api.semanticscholar.org/graph/v1/paper/DOI:"+paper_doi+"?fields=referenceCount,citationCount,influentialCitationCount,externalIds,authors.hIndex,authors.affiliations,authors.paperCount,fieldsOfStudy,publicationTypes,publicationDate,journal,citations.authors,citations.fieldsOfStudy,citations.externalIds,citations.s2FieldsOfStudy")
# req = requests.get("https://api.semanticscholar.org/graph/v1/paper/DOI:"+paper_doi+"?fields=referenceCount,citationCount,influentialCitationCount,fieldsOfStudy,publicationDate,authors,authors.name,authors.hIndex,externalIds")

if req.status_code == 200:
  entries = req.json()

  ss_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in entries.items() ]));

else:
  print("Error occured while requesting data from the Semantic Scholar Academic Graph API endpoint.")


In [1677]:
ss_df

,paperId,externalIds,referenceCount,citationCount,influentialCitationCount,fieldsOfStudy,publicationTypes,publicationDate,journal,authors,citations
0,b0c065cd43aa7280e766b5dcbcc7e26abce59330,NaN,73.0,10024.0,1482.0,Computer Science,JournalArticle,2015-11-02,NaN,"{'authorId': '2442177', 'affiliations': [], 'paperCount': 45, 'hIndex': 20}","{'paperId': '9424b5bc9a631b66df372ba6b1bab549676c276c', 'externalIds': {'DOI': '10.1016/j.optlaseng.2022.107409', 'CorpusId': 254621382}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [], 'authors': [{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]}"
1,NaN,NaN,NaN,NaN,NaN,Medicine,NaN,NaN,NaN,"{'authorId': '47645184', 'affiliations': [], 'paperCount': 64, 'hIndex': 20}","{'paperId': 'ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104437', 'CorpusId': 254561039}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 's2-fos-model'}], 'authors': [{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]}"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'authorId': '1745672', 'affiliations': [], 'paperCount': 490, 'hIndex': 84}","{'paperId': '325bf3e8cc37a023e2fb8809bf356e74b4d0a5da', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104430', 'CorpusId': 254387365}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Medicine', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]}"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paperId': 'c57d615133f53153e483f02cc3218d0fa630f50e', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104431', 'CorpusId': 254388253}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Physics', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]}"
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paperId': 'f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104426', 'CorpusId': 254301588}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]}"
...,...,...,...,...,...,...,...,...,...,...,...
MAG,NaN,2952940720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PubMed,NaN,28060704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IEEE Transactions on Pattern Analysis and Machine Intelligence,NaN,NaN
pages,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2481-2495,NaN,NaN


In [1678]:
ss_df['DOI'] = ss_df.loc['DOI'].externalIds
ss_df['CorpusID'] = ss_df.loc['CorpusId'].externalIds
ss_df['paperId'] = ss_df['paperId'].iloc[0]
ss_df = ss_df[ss_df['externalIds'].isna()]
ss_df['journal_name'] = ss_df.loc['name'].journal

C:\Users\Maurice\AppData\Local\Temp\ipykernel_5436\1785097148.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ss_df['journal_name'] = ss_df.loc['name'].journal


In [1679]:
citations_df = ss_df[~ss_df['citations'].isna()][['DOI', 'CorpusID', 'paperId', 'citations']]
ss_df = ss_df[~ss_df['authors'].isna()][['DOI', 'CorpusID', 'paperId', 'referenceCount', 'citationCount', 'influentialCitationCount', 'fieldsOfStudy', 'journal_name','publicationDate', 'authors']]

In [1680]:
citations_df

,DOI,CorpusID,paperId,citations
0,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': '9424b5bc9a631b66df372ba6b1bab549676c276c', 'externalIds': {'DOI': '10.1016/j.optlaseng.2022.107409', 'CorpusId': 254621382}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [], 'authors': [{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]}"
1,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104437', 'CorpusId': 254561039}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 's2-fos-model'}], 'authors': [{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]}"
2,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': '325bf3e8cc37a023e2fb8809bf356e74b4d0a5da', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104430', 'CorpusId': 254387365}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Medicine', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]}"
3,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'c57d615133f53153e483f02cc3218d0fa630f50e', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104431', 'CorpusId': 254388253}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Physics', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]}"
4,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104426', 'CorpusId': 254301588}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]}"
...,...,...,...,...
995,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'cb534c72f2d7f5447176406534d0bda7f8c26447', 'externalIds': {'DOI': '10.1007/s00603-022-02911-x', 'CorpusId': 248991543}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Geology', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2166070414', 'name': 'Ke Ma'}, {'authorId': '2166105751', 'name': 'Xingye Sun'}, {'authorId': '2148906270', 'name': 'Zhenghu Zhang'}, {'authorId': '2166084917', 'name': 'Jing Hu'}, {'authorId': '9882310', 'name': 'Zuorong Wang'}]}"
996,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': '8cee11ee1ff27747e93aedc70c70582399f0f0dd', 'externalIds': {'ArXiv': '2205.10840', 'CorpusId': 250627672}, 'fiel

In [1681]:
ss_df

,DOI,CorpusID,paperId,referenceCount,citationCount,influentialCitationCount,fieldsOfStudy,journal_name,publicationDate,authors
0,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,73.0,10024.0,1482.0,Computer Science,IEEE Transactions on Pattern Analysis and Machine Intelligence,2015-11-02,"{'authorId': '2442177', 'affiliations': [], 'paperCount': 45, 'hIndex': 20}"
1,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,NaN,NaN,NaN,Medicine,IEEE Transactions on Pattern Analysis and Machine Intelligence,NaN,"{'authorId': '47645184', 'affiliations': [], 'paperCount': 64, 'hIndex': 20}"
2,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,NaN,NaN,NaN,NaN,IEEE Transactions on Pattern Analysis and Machine Intelligence,NaN,"{'authorId': '1745672', 'affiliations': [], 'paperCount': 490, 'hIndex': 84}"


In [1682]:
ss_df['hIndex'] = ss_df['authors'].apply(lambda x: x['hIndex'])

In [1683]:
ss_df

,DOI,CorpusID,paperId,referenceCount,citationCount,influentialCitationCount,fieldsOfStudy,journal_name,publicationDate,authors,hIndex
0,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,73.0,10024.0,1482.0,Computer Science,IEEE Transactions on Pattern Analysis and Machine Intelligence,2015-11-02,"{'authorId': '2442177', 'affiliations': [], 'paperCount': 45, 'hIndex': 20}",20
1,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,NaN,NaN,NaN,Medicine,IEEE Transactions on Pattern Analysis and Machine Intelligence,NaN,"{'authorId': '47645184', 'affiliations': [], 'paperCount': 64, 'hIndex': 20}",20
2,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,NaN,NaN,NaN,NaN,IEEE Transactions on Pattern Analysis and Machine Intelligence,NaN,"{'authorId': '1745672', 'affiliations': [], 'paperCount': 490, 'hIndex': 84}",84


In [1684]:
ss_df['mean_hIndex'] = ss_df['hIndex'].mean().round(0).astype(int)
ss_df['max_hIndex'] = ss_df['hIndex'].max().round(0).astype(int)
ss_df['min_hIndex'] = ss_df['hIndex'].min().round(0).astype(int)

In [1685]:
ss_df = ss_df.iloc[0][['DOI', 'CorpusID', 'paperId', 'publicationDate', 'fieldsOfStudy', 'referenceCount', 'citationCount', 'influentialCitationCount', 'mean_hIndex', 'max_hIndex', 'min_hIndex']]

In [1686]:
ss_df['referenceCount'] = ss_df['referenceCount'].astype(int)
ss_df['citationCount'] = ss_df['citationCount'].astype(int)
ss_df['influentialCitationCount'] = ss_df['influentialCitationCount'].astype(int)

In [1687]:
ss_df

DOI                                       10.1109/TPAMI.2016.2644615
CorpusID                                                    60814714
paperId                     b0c065cd43aa7280e766b5dcbcc7e26abce59330
publicationDate                                           2015-11-02
fieldsOfStudy                                       Computer Science
referenceCount                                                    73
citationCount                                                  10024
influentialCitationCount                                        1482
mean_hIndex                                                       41
max_hIndex                                                        84
min_hIndex                                                        20
Name: 0, dtype: object

### Citations DF

In [1688]:
citations_df.rename(columns={'DOI': 'REF_DOI', 'CorpusID': 'REF_CorpusID', 'paperId': 'REF_paperID'}, inplace=True)

In [1689]:
citations_df.head(5)

,REF_DOI,REF_CorpusID,REF_paperID,citations
0,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': '9424b5bc9a631b66df372ba6b1bab549676c276c', 'externalIds': {'DOI': '10.1016/j.optlaseng.2022.107409', 'CorpusId': 254621382}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [], 'authors': [{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]}"
1,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104437', 'CorpusId': 254561039}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 's2-fos-model'}], 'authors': [{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]}"
2,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': '325bf3e8cc37a023e2fb8809bf356e74b4d0a5da', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104430', 'CorpusId': 254387365}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Medicine', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]}"
3,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'c57d615133f53153e483f02cc3218d0fa630f50e', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104431', 'CorpusId': 254388253}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Physics', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]}"
4,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,"{'paperId': 'f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef', 'externalIds': {'DOI': '10.1016/j.bspc.2022.104426', 'CorpusId': 254301588}, 'fieldsOfStudy': None, 's2FieldsOfStudy': [{'category': 'Computer Science', 'source': 's2-fos-model'}], 'authors': [{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]}"


In [1690]:
temp_citation_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in citations_df['citations'].items() ])).transpose()

In [1691]:
temp_citation_df

,paperId,externalIds,fieldsOfStudy,s2FieldsOfStudy,authors
0,9424b5bc9a631b66df372ba6b1bab549676c276c,"{'DOI': '10.1016/j.optlaseng.2022.107409', 'CorpusId': 254621382}",None,[],"[{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]"
1,ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b,"{'DOI': '10.1016/j.bspc.2022.104437', 'CorpusId': 254561039}",None,"[{'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]"
2,325bf3e8cc37a023e2fb8809bf356e74b4d0a5da,"{'DOI': '10.1016/j.bspc.2022.104430', 'CorpusId': 254387365}",None,"[{'category': 'Medicine', 'source': 's2-fos-model'}]","[{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]"
3,c57d615133f53153e483f02cc3218d0fa630f50e,"{'DOI': '10.1016/j.bspc.2022.104431', 'CorpusId': 254388253}",None,"[{'category': 'Physics', 'source': 's2-fos-model'}]","[{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]"
4,f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef,"{'DOI': '10.1016/j.bspc.2022.104426', 'CorpusId': 254301588}",None,"[{'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]"
...,...,...,...,...,...
995,cb534c72f2d7f5447176406534d0bda7f8c26447,"{'DOI': '10.1007/s00603-022-02911-x', 'CorpusId': 248991543}",None,"[{'category': 'Geology', 'source': 's2-fos-model'}]","[{'authorId': '2166070414', 'name': 'Ke Ma'}, {'authorId': '2166105751', 'name': 'Xingye Sun'}, {'authorId': '2148906270', 'name': 'Zhenghu Zhang'}, {'authorId': '2166084917', 'name': 'Jing Hu'}, {'authorId': '9882310', 'name': 'Zuorong Wang'}]"
996,8cee11ee1ff27747e93aedc70c70582399f0f0dd,"{'ArXiv': '2205.10840', 'CorpusId': 250627672}",[Computer Science],"[{'category': 'Computer Science', 'source': 'external'}, {'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
997,c91acbf2022920c01842c9e42d6fc3e7cf97a901,"{'DBLP': 'journals/corr/abs-2205-10840', 'DOI': '10.48550/arXiv.2205.10840', 'CorpusId': 248987551}",[Computer Science],"[{'category': 'Computer Science', 'source': 'external'}, {'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
998,9fd889980690d5b2cc102fe6825844e72fff07c9,"{'DBLP': 'journals/corr/abs-2205-09949', 'ArXiv': '2205.09949', 'DOI': '10.48550/arXiv.2205.09949', 'CorpusId': 248965269}",[Computer Science],"[{'category': 'Computer Science', 'source': 'external'}, {'category': 

In [1692]:
temp_citation_df = temp_citation_df.join(pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_citation_df['externalIds'].items() ])).transpose())[['paperId', 'DOI', 'CorpusId', 'fieldsOfStudy', 's2FieldsOfStudy', 'authors']]

In [1693]:
temp_citation_df

,paperId,DOI,CorpusId,fieldsOfStudy,s2FieldsOfStudy,authors
0,9424b5bc9a631b66df372ba6b1bab549676c276c,10.1016/j.optlaseng.2022.107409,254621382,None,[],"[{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]"
1,ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b,10.1016/j.bspc.2022.104437,254561039,None,"[{'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]"
2,325bf3e8cc37a023e2fb8809bf356e74b4d0a5da,10.1016/j.bspc.2022.104430,254387365,None,"[{'category': 'Medicine', 'source': 's2-fos-model'}]","[{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]"
3,c57d615133f53153e483f02cc3218d0fa630f50e,10.1016/j.bspc.2022.104431,254388253,None,"[{'category': 'Physics', 'source': 's2-fos-model'}]","[{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]"
4,f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef,10.1016/j.bspc.2022.104426,254301588,None,"[{'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]"
...,...,...,...,...,...,...
995,cb534c72f2d7f5447176406534d0bda7f8c26447,10.1007/s00603-022-02911-x,248991543,None,"[{'category': 'Geology', 'source': 's2-fos-model'}]","[{'authorId': '2166070414', 'name': 'Ke Ma'}, {'authorId': '2166105751', 'name': 'Xingye Sun'}, {'authorId': '2148906270', 'name': 'Zhenghu Zhang'}, {'authorId': '2166084917', 'name': 'Jing Hu'}, {'authorId': '9882310', 'name': 'Zuorong Wang'}]"
996,8cee11ee1ff27747e93aedc70c70582399f0f0dd,NaN,250627672,[Computer Science],"[{'category': 'Computer Science', 'source': 'external'}, {'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
997,c91acbf2022920c01842c9e42d6fc3e7cf97a901,10.48550/arXiv.2205.10840,248987551,[Computer Science],"[{'category': 'Computer Science', 'source': 'external'}, {'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
998,9fd889980690d5b2cc102fe6825844e72fff07c9,10.48550/arXiv.2205.09949,248965269,[Computer Science],"[{'category': 'Computer Science', 'source': 'external'}, {'category': 'Computer Science', 'source': 's2-fos-model'}]","[{'authorId': '153832589', 'name': 'Teppei Suzuki'}]"


In [1694]:
# pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in x_df['0'].items() ])).transpose().items() ])).transpose()
temp_FoS_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_citation_df['s2FieldsOfStudy'].items() ])).transpose()

C:\Users\Maurice\AppData\Local\Temp\ipykernel_5436\1357937721.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  temp_FoS_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_citation_df['s2FieldsOfStudy'].items() ])).transpose()


In [1695]:
temp_FoS_df

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,"{'category': 'Computer Science', 'source': 's2-fos-model'}",NaN,NaN,NaN
2,"{'category': 'Medicine', 'source': 's2-fos-model'}",NaN,NaN,NaN
3,"{'category': 'Physics', 'source': 's2-fos-model'}",NaN,NaN,NaN
4,"{'category': 'Computer Science', 'source': 's2-fos-model'}",NaN,NaN,NaN
...,...,...,...,...
995,"{'category': 'Geology', 'source': 's2-fos-model'}",NaN,NaN,NaN
996,"{'category': 'Computer Science', 'source': 'external'}","{'category': 'Computer Science', 'source': 's2-fos-model'}",NaN,NaN
997,"{'category': 'Computer Science', 'source': 'external'}","{'category': 'Computer Science', 'source': 's2-fos-model'}",NaN,NaN
998,"{'category': 'Computer Science', 'source': 'external'}","{'category': 'Computer Science', 'source': 's2-fos-model'}",NaN,NaN


In [1696]:
if len(temp_FoS_df.columns) > 1:
    for idx, col in enumerate(temp_FoS_df.columns):
        if idx==0:
            s2_FoS_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_FoS_df[0].items() ])).transpose()[['category']].rename(columns={'category': 'category_0'})
        else:
            s2_FoS_df = s2_FoS_df.join(pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in temp_FoS_df[idx].items() ])).transpose()[['category']]).rename(columns={'category': 'category_'+str(idx)})

In [1697]:
s2_FoS_df.fillna('', inplace=True)

In [1698]:
def extract_s2_FieldsOfStudy(row):
    s2_FieldsOfStudy_List = []

    for el in list(dict.fromkeys(row)):
        if el:
            s2_FieldsOfStudy_List.append(el)

    # if len(s2_FieldsOfStudy_List) == 0:
    #     return ''
    # else:
    #     return s2_FieldsOfStudy_List

    return s2_FieldsOfStudy_List

In [1699]:
temp_FoS_df['s2_FieldsOfStudy'] = s2_FoS_df.apply(lambda row: extract_s2_FieldsOfStudy(row), axis=1)

In [1700]:
temp_citation_df = temp_citation_df.join(temp_FoS_df.iloc[:,-1:])[['paperId', 'DOI', 'CorpusId', 'fieldsOfStudy', 's2_FieldsOfStudy', 'authors']].fillna(np.nan)

In [1701]:
temp_citation_df

,paperId,DOI,CorpusId,fieldsOfStudy,s2_FieldsOfStudy,authors
0,9424b5bc9a631b66df372ba6b1bab549676c276c,10.1016/j.optlaseng.2022.107409,254621382,NaN,[],"[{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]"
1,ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b,10.1016/j.bspc.2022.104437,254561039,NaN,[Computer Science],"[{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]"
2,325bf3e8cc37a023e2fb8809bf356e74b4d0a5da,10.1016/j.bspc.2022.104430,254387365,NaN,[Medicine],"[{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]"
3,c57d615133f53153e483f02cc3218d0fa630f50e,10.1016/j.bspc.2022.104431,254388253,NaN,[Physics],"[{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]"
4,f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef,10.1016/j.bspc.2022.104426,254301588,NaN,[Computer Science],"[{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]"
...,...,...,...,...,...,...
995,cb534c72f2d7f5447176406534d0bda7f8c26447,10.1007/s00603-022-02911-x,248991543,NaN,[Geology],"[{'authorId': '2166070414', 'name': 'Ke Ma'}, {'authorId': '2166105751', 'name': 'Xingye Sun'}, {'authorId': '2148906270', 'name': 'Zhenghu Zhang'}, {'authorId': '2166084917', 'name': 'Jing Hu'}, {'authorId': '9882310', 'name': 'Zuorong Wang'}]"
996,8cee11ee1ff27747e93aedc70c70582399f0f0dd,NaN,250627672,[Computer Science],[Computer Science],"[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
997,c91acbf2022920c01842c9e42d6fc3e7cf97a901,10.48550/arXiv.2205.10840,248987551,[Computer Science],[Computer Science],"[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
998,9fd889980690d5b2cc102fe6825844e72fff07c9,10.48550/arXiv.2205.09949,248965269,[Computer Science],[Computer Science],"[{'authorId': '153832589', 'name': 'Teppei Suzuki'}]"


In [1702]:
def consolidate_FoS(FoS, s2FoS):
    if FoS == '' & s2FoS != '':
        return s2FoS
    elif FoS != '' & s2FoS == '':
        return FoS
    elif FoS != '' & s2FoS == '':
        return FoS+s2FoS
    else:
        return []

In [1703]:
temp_citation_df['fieldsOfStudy'] = [ [] if x is np.NaN else x for x in temp_citation_df['fieldsOfStudy'] ]

In [1704]:
temp_citation_df

,paperId,DOI,CorpusId,fieldsOfStudy,s2_FieldsOfStudy,authors
0,9424b5bc9a631b66df372ba6b1bab549676c276c,10.1016/j.optlaseng.2022.107409,254621382,NaN,[],"[{'authorId': '2116430856', 'name': 'Suqin Wang'}, {'authorId': '2195767616', 'name': 'Taiqin Chen'}, {'authorId': '2087816711', 'name': 'Min Shi'}, {'authorId': None, 'name': 'Dengmin Zhu'}, {'authorId': None, 'name': 'Jia Wang'}]"
1,ccd8e6ebaba5c1edf78ae209dbb9c1b9f262d24b,10.1016/j.bspc.2022.104437,254561039,NaN,[Computer Science],"[{'authorId': '1411370729', 'name': 'Xiwang Xie'}, {'authorId': '40538943', 'name': 'Weidong Zhang'}, {'authorId': '2115622662', 'name': 'Xipeng Pan'}, {'authorId': '2088588647', 'name': 'Lijie Xie'}, {'authorId': '2166597539', 'name': 'Feng Shao'}, {'authorId': '50771379', 'name': 'Wenyi Zhao'}, {'authorId': '3078465', 'name': 'Jubai An'}]"
2,325bf3e8cc37a023e2fb8809bf356e74b4d0a5da,10.1016/j.bspc.2022.104430,254387365,NaN,[Medicine],"[{'authorId': '2111214180', 'name': 'Yang Deng'}, {'authorId': '1691325766', 'name': 'L. Lan'}, {'authorId': '2054505448', 'name': 'Lei You'}, {'authorId': '2181072758', 'name': 'Kai Chen'}, {'authorId': '2117699190', 'name': 'Li Peng'}, {'authorId': '2452732', 'name': 'Weiling Zhao'}, {'authorId': '2113347820', 'name': 'Bin Song'}, {'authorId': '2154461295', 'name': 'Yi Wang'}, {'authorId': '46932056', 'name': 'Zhiwei Ji'}, {'authorId': '2155797568', 'name': 'Xiaobo Zhou'}]"
3,c57d615133f53153e483f02cc3218d0fa630f50e,10.1016/j.bspc.2022.104431,254388253,NaN,[Physics],"[{'authorId': '2053108760', 'name': 'Wenchao Cui'}, {'authorId': '40561809', 'name': 'Dan Meng'}, {'authorId': '2070275382', 'name': 'Ke Lu'}, {'authorId': '2115853393', 'name': 'Yirong Wu'}, {'authorId': '145244210', 'name': 'Zhihong Pan'}, {'authorId': '2180533787', 'name': 'Xiaolong Li'}, {'authorId': '144408598', 'name': 'Shuifa Sun'}]"
4,f09a7e03542a3d7b8bcf4b28cbf3c640eac64cef,10.1016/j.bspc.2022.104426,254301588,NaN,[Computer Science],"[{'authorId': '2108987750', 'name': 'Jianyong Li'}, {'authorId': '2152151877', 'name': 'Ge Gao'}, {'authorId': '2108712846', 'name': 'Lei Yang'}, {'authorId': '2116412921', 'name': 'Yanhong Liu'}]"
...,...,...,...,...,...,...
995,cb534c72f2d7f5447176406534d0bda7f8c26447,10.1007/s00603-022-02911-x,248991543,NaN,[Geology],"[{'authorId': '2166070414', 'name': 'Ke Ma'}, {'authorId': '2166105751', 'name': 'Xingye Sun'}, {'authorId': '2148906270', 'name': 'Zhenghu Zhang'}, {'authorId': '2166084917', 'name': 'Jing Hu'}, {'authorId': '9882310', 'name': 'Zuorong Wang'}]"
996,8cee11ee1ff27747e93aedc70c70582399f0f0dd,NaN,250627672,[Computer Science],[Computer Science],"[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
997,c91acbf2022920c01842c9e42d6fc3e7cf97a901,10.48550/arXiv.2205.10840,248987551,[Computer Science],[Computer Science],"[{'authorId': '2153464492', 'name': 'Arnaud Deleruyelle'}, {'authorId': '2375903', 'name': 'Cristian Versari'}, {'authorId': '2087030915', 'name': 'J. Klein'}]"
998,9fd889980690d5b2cc102fe6825844e72fff07c9,10.48550/arXiv.2205.09949,248965269,[Computer Science],[Computer Science],"[{'authorId': '153832589', 'name': 'Teppei Suzuki'}]"


In [1705]:
temp_citation_df.fillna('', inplace=True)

In [1706]:
# List comprehension: Ersetze alle Missing Values mit einer leeren Liste, um diese in einem weiteren Schritt mit s2_FieldsOfStudy zu konkatenieren
temp_citation_df['fieldsOfStudy'] = [ [] if x == '' else x for x in temp_citation_df['fieldsOfStudy'] ]

In [1707]:
temp_citation_df['fieldsOfStudy'] = temp_citation_df['fieldsOfStudy']+temp_citation_df['s2_FieldsOfStudy']

In [1708]:
temp_citation_df['fieldsOfStudy'] = temp_citation_df['fieldsOfStudy'].apply(lambda row: list(dict.fromkeys(row)))

In [1709]:
temp_citation_df.drop(['s2_FieldsOfStudy'], axis=1, inplace=True)

In [1710]:
citations_df = citations_df.join(temp_citation_df)

In [1711]:
del temp_citation_df, temp_FoS_df, s2_FoS_df

In [1712]:
ss_completed_flag = True

# PyAltmetric

In [1713]:
from pyaltmetric import Altmetric

a = Altmetric()

altmetric_json_data = a.doi(paper_doi)

In [1714]:
altmetric_json_data

{'title': 'SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation',
 'doi': '10.1109/tpami.2016.2644615',
 'pmid': '28060704',
 'altmetric_jid': '4f6fa4ed3cf058f610002a67',
 'issns': ['0162-8828', '2160-9292', '1939-3539'],
 'journal': 'IEEE Transactions on Software Engineering',
 'cohorts': {'pub': 4},
 'abstract': 'We present a novel and practical deep fully convolutional neural network architecture for semantic pixel-wise segmentation termed SegNet. This core trainable segmentation engine consists of an encoder network, a corresponding decoder network followed by a pixel-wise classification layer. The architecture of the encoder network is topologically identical to the 13 convolutional layers in the VGG16 network [1]. The role of the decoder network is to map the low resolution encoder feature maps to full input resolution feature maps for pixel-wise classification. The novelty of SegNet lies is in the manner in which the decoder upsamples its lower resolu

In [1715]:
am_df = pd.DataFrame.from_dict(altmetric_json_data.items()).transpose()

In [1716]:
am_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
0,title,doi,pmid,altmetric_jid,issns,journal,cohorts,abstract,context,authors,type,pubdate,altmetric_id,schema,is_oa,publisher_subjects,cited_by_posts_count,cited_by_policies_count,cited_by_tweeters_count,cited_by_patents_count,cited_by_accounts_count,last_updated,score,history,url,added_on,published_on,subjects,scopus_subjects,readers,readers_count,images,details_url
1,SegNet: A Deep Convolutional Encoder-Decoder Architecture for Image Segmentation,10.1109/tpami.2016.2644615,28060704,4f6fa4ed3cf058f610002a67,"[0162-8828, 2160-9292, 1939-3539]",IEEE Transactions on Software Engineering,{'pub': 4},"We present a novel and practical deep fully convolutional neural network architecture for semantic pixel-wise segmentation termed SegNet. This core trainable segmentation engine consists of an encoder network, a corresponding decoder network followed by a pixel-wise classification layer. The architecture of the encoder network is topologically identical to the 13 convolutional layers in the VGG16 network [1]. The role of the decoder network is to map the low resolution encoder feature maps to full input resolution feature maps for pixel-wise classification. The novelty of SegNet lies is in the manner in which the decoder upsamples its lower resolution input feature map(s). Specifically, the decoder uses pooling indices computed in the max-pooling step of the corresponding encoder to perform non-linear upsampling. This eliminates the need for learning to upsample. The upsampled maps are sparse and are then convolved with trainable filters to produce dense feature maps. We compare our proposed architecture with the widely adopted FCN [2] and also with the well known DeepLab-LargeFOV [3], DeconvNet [4] architectures. This comparison reveals the memory versus accuracy trade-off involved in achieving good segmentation performance. SegNet was primarily motivated by scene understanding applications. Hence, it is designed to be efficient both in terms of memory and computational time during inference. It is also significantly smaller in the number of trainable parameters than other competing architectures and can be trained end-to-end using stochastic gradient descent. We also performed a controlled benchmark of SegNet and other architectures on both road scenes and SUN RGB-D indoor scene segmentation tasks. These quantitative assessments show that SegNet provides good performance with competitive inference time and most efficient inference memory-wise as compared to other architectures. We also provide a Caffe implementation of SegNet and a web demo at http://mi.eng.cam.ac.uk/projects/segnet/.","{'all': {'count': 22551053, 'mean': 10.140544749649461, 'rank': 1328842, 'pct': 94, 'higher_than': 21219949}, 'journal': {'count': 6013, 'mean': 4.531498420089805, 'rank': 78, 'pct': 98, 'higher_than': 5935}, 'similar_age_3m': {'count': 451334, 'mean': 12.412878560001282, 'rank': 37216, 'pct': 91, 'higher_than': 414054}, 'similar_age_journal_3m': {'count': 53, 'mean': 4.218075471698113, 'rank': 2, 'pct': 96, 'higher_than': 51}}","[Vijay Badrinarayanan, Alex Kendall, Roberto Cipolla]",article,1512086400,28320951,1.5.4,False,"[{'name': 'Computer Software', 'scheme': 'era'}, {'name': 'Information Systems', 'scheme': 'era'}]",52,1,4,46,51,1666673806,23.5,"{'1y': 13, '6m': 10, '3m': 7, '1m': 0, '1w': 0, '6d': 0, '5d': 0, '4d': 0, '3d': 0, '2d': 0, '1d': 0, 'at': 23.5}",http://dx.doi.org/10.1109/tpami.2016.2644615,1509665025,1512086400,[medicalinformatics],"[Physical Sciences, Computer Science]","{'citeulike': '1', 'mendeley': '6442', 'connotea': '0'}",6438,"{'small': 'https://badges.altmetric.com/?size=64&score=24&types=aaaaaatd', 'medium': 'https://badges.altmetric.com/?size=100&score=24&types=aaaaaatd', 'large': 'https://badges.altmetric.com/?size=180&score=24&types=aaaaaatd'}",https://www.altmetric.com/details.php?citation_id=28320951


In [1717]:
am_df.columns = am_df.iloc[0]
# am_df.rename(columns={'doi': 'DOI', 'score': 'altmetric_attention_score', 'cited_by_accounts_count': 'altmetric_cited_by_accounts_count', 'cited_by_policies_count': 'altmetric_cited_by_policies_count', 'cited_by_posts_count': 'altmetric_cited_by_posts_count', 'cited_by_tweeters_count': 'altmetric_cited_by_tweeters_count', 'cited_by_feeds_count': 'altmetric_cited_by_feeds_count', 'readers_count': 'altmetric_readers_count'}, inplace=True)
am_df = am_df.iloc[1]
am_df['score'] = np.ceil(am_df['score']).astype(int)

In [1718]:
col_list = []
for col in am_df.keys():
    if any(x in col for x in ['count', 'score']):
        col_list.append(col)

In [1719]:
am_df = am_df[col_list]

In [1720]:
for col in am_df.keys():
    if col in col_list:
        am_df.rename({col:'altmetric_'+col}, inplace=True)

In [1721]:
# am_df['DOI'] = am_df['DOI'].upper()

In [1722]:
am_completed_flag = True

# Concatenate SS & AM

In [1723]:
pd.concat([ss_df, am_df])

DOI                                                10.1109/TPAMI.2016.2644615
CorpusID                                                             60814714
paperId                              b0c065cd43aa7280e766b5dcbcc7e26abce59330
publicationDate                                                    2015-11-02
fieldsOfStudy                                                Computer Science
referenceCount                                                             73
citationCount                                                           10024
influentialCitationCount                                                 1482
mean_hIndex                                                                41
max_hIndex                                                                 84
min_hIndex                                                                 20
altmetric_cited_by_posts_count                                             52
altmetric_cited_by_policies_count                               

In [1724]:
pd.DataFrame.from_dict(pd.concat([ss_df, am_df])).transpose()

,DOI,CorpusID,paperId,publicationDate,fieldsOfStudy,referenceCount,citationCount,influentialCitationCount,mean_hIndex,max_hIndex,min_hIndex,altmetric_cited_by_posts_count,altmetric_cited_by_policies_count,altmetric_cited_by_tweeters_count,altmetric_cited_by_patents_count,altmetric_cited_by_accounts_count,altmetric_score,altmetric_readers_count
0,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,2015-11-02,Computer Science,73,10024,1482,41,84,20,52,1,4,46,51,24,6438


# IEEE Xplore

In [1725]:
xplore_api_key = 'ydjxbh5jv9ba3cqhhg8wm82p'

In [1726]:
req = requests.get("http://ieeexploreapi.ieee.org/api/v1/search/articles?apikey="+xplore_api_key+"&format=json&max_records=25&start_record=1&sort_order=asc&sort_field=article_number&doi="+paper_doi)

if req.status_code == 200:
  entries = req.json()

  ieee_df=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in entries.items() ]));


In [1727]:
ieee_df

total_records  total_searched  \
0              1         5812518   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [1728]:
if ieee_df['total_records'].iloc[0]>0:
    # ieee_df['DOI'] = ieee_df['articles'].apply(lambda x: x['doi'])
    ieee_df['ieee_citing_paper_count'] = ieee_df['articles'].apply(lambda x: x['citing_paper_count'])
    ieee_df['ieee_citing_patent_count'] = ieee_df['articles'].apply(lambda x: x['citing_patent_count'])

    ieee_df = ieee_df.iloc[0,-2:]

    ieee_completed_flag = True

# Concat SS & AM & IEEE

In [1729]:
if ss_completed_flag & am_completed_flag & ieee_completed_flag:
    final_df = pd.concat([ss_df, am_df, ieee_df])
elif ss_completed_flag & am_completed_flag & ~ieee_completed_flag:
    final_df = pd.concat([ss_df, am_df])
else:
    final_df = pd.concat([ss_df])

In [1730]:
final_df

DOI                                                10.1109/TPAMI.2016.2644615
CorpusID                                                             60814714
paperId                              b0c065cd43aa7280e766b5dcbcc7e26abce59330
publicationDate                                                    2015-11-02
fieldsOfStudy                                                Computer Science
referenceCount                                                             73
citationCount                                                           10024
influentialCitationCount                                                 1482
mean_hIndex                                                                41
max_hIndex                                                                 84
min_hIndex                                                                 20
altmetric_cited_by_posts_count                                             52
altmetric_cited_by_policies_count                               

In [1731]:
pd.DataFrame.from_dict(final_df).transpose()

,DOI,CorpusID,paperId,publicationDate,fieldsOfStudy,referenceCount,citationCount,influentialCitationCount,mean_hIndex,max_hIndex,min_hIndex,altmetric_cited_by_posts_count,altmetric_cited_by_policies_count,altmetric_cited_by_tweeters_count,altmetric_cited_by_patents_count,altmetric_cited_by_accounts_count,altmetric_score,altmetric_readers_count,ieee_citing_paper_count,ieee_citing_patent_count
0,10.1109/TPAMI.2016.2644615,60814714,b0c065cd43aa7280e766b5dcbcc7e26abce59330,2015-11-02,Computer Science,73,10024,1482,41,84,20,52,1,4,46,51,24,6438,7501,0
